In [57]:
import numpy as np
import scipy as sp
import scipy.stats as stats

import matplotlib.pyplot as plt
%matplotlib inline

import torch 
import torchvision
from torch.autograd import Variable
import torch.utils.data as data
print("DONE")

DONE


Please train the network on the data from file classification_problem.npy. File can be loaded using np.load() function. It contains 3000 rows. First 1000 rows coressponds to class 0, next 1000 to class1 and last 1000 rows to class 2.#%%

In [58]:
size = 3000
data = np.load('classification_problem.npy')
data_reshape =  np.hstack((data,np.zeros((data.shape[0],1))))
for dd in range(size):
    data_reshape[dd] = np.append(data[dd],[np.long(dd/1000)+1])
print("Data reshaped")

Data reshaped


In [59]:
x_train = torch.from_numpy(data)
y_train = torch.from_numpy(data_reshape[:,12:13])
print(data_reshape.shape)
print(x_train.shape)
print(y_train.shape)

(3000, 13)
torch.Size([3000, 12])
torch.Size([3000, 1])


In [60]:
n_in = 12
n_hidden = 128
n_out = 4
training_model = \
    torch.nn.Sequential(torch.nn.Linear(n_in,n_hidden,bias=True),
                        torch.nn.ReLU(),
                        torch.nn.Linear(n_hidden,n_out),
                        torch.nn.LogSoftmax(dim=1))
print("Model created")

Model created


In [61]:
valid_size = 200
dataset = torch.utils.data.TensorDataset(x_train, y_train)
(train_set, valid_set) = torch.utils.data.random_split(dataset,(size-valid_size,valid_size))
train_loader = torch.utils.data.DataLoader(train_set,batch_size=valid_size,shuffle=True)
print(valid_size)

200


In [67]:
optimizer = torch.optim.Adam(training_model.parameters(),lr =0.001)
loss_func = torch.nn.NLLLoss()
max_epoch = 1000
loss = 0
err_train=[]
err_valid=[]
valid_data = Variable(valid_set[:][0],requires_grad=False)
valid_target = Variable(valid_set[:][1].long(),requires_grad=False)
for epoch in range(max_epoch):
    for datum in train_loader:
        optimizer.zero_grad()
        (features,target1) = datum
        data = Variable(features,requires_grad=False)
        target = Variable(target1.long(),requires_grad=False)
        pred = training_model(features)
        loss = loss_func(pred, target.view(-1))
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            vpred  = training_model(valid_data)
            vloss  = loss_func(vpred,valid_target.view(-1))
            err_valid.append(vloss)
            pred  = training_model(data)
            loss  = loss_func(pred,target.view(-1))
            err_train.append(loss)
    if epoch%100 == 0:
        print("epoch %d %f %f " % (epoch, loss, vloss))

epoch 0 1.098546 1.099597 
epoch 100 1.098482 1.098944 
epoch 200 1.099067 1.098940 
epoch 300 1.098697 1.098837 
epoch 400 1.098772 1.098894 
epoch 500 1.098494 1.098950 
epoch 600 1.098379 1.098840 
epoch 700 1.098804 1.098975 
epoch 800 1.098407 1.098878 
epoch 900 1.098861 1.098917 
